# Section 2  - Homework
**Author:** Luis Henrique Velasquez

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command mlflow --version and check the output.

What's the version that you have?

In [34]:
import mlflow
print(mlflow.__version__)

2.14.1


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7

In [8]:
import os
itens = os.listdir('output/')
total_files = sum(1 for item in itens if os.path.isfile(os.path.join('output/', item)))
print(f'Total files in "output" folder: {total_files}')

Total files in "output" folder: 4


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder [`homework`](homework). 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

* 2
* 4
* 8
* 10

In [16]:
run_id = "62f1d72e7ba2403d8d2bc2cd073ba3e4"
run = mlflow.get_run(run_id)
params = run.data.params
params['min_samples_split']

'2'

## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`

In [17]:
# mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Response: "--default-artifact-root"

## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* 5.335
* 5.818
* 6.336

In [19]:
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# Checking the experiments present in MLFlow:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/mlops-zoomcamp/02-experiment-tracking/homework/artifacts_local/1', creation_time=1721416466872, experiment_id='1', last_update_time=1721416466872, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/mlops-zoomcamp/02-experiment-tracking/homework/artifacts_local/0', creation_time=1721415947478, experiment_id='0', last_update_time=1721415947478, lifecycle_stage='active', name='Default', tags={}>]

In [25]:
# Searching for the best runs for the experiment with ID '1' (random-forest-hyperopt):
runs = client.search_runs(
    experiment_ids='1',
    run_view_type=ViewType.ACTIVE_ONLY, # Considering only active runs
    max_results=1, # Maximum number of results presented
    order_by=["metrics.rmse ASC"] # Sorting the results in order of RMSE
)

# Presenting the runs and their respective RMSE's
for run in runs:
    print("Run with the best (lower) RMSE: \n " +
          f"run id: {run.info.run_id}, \n " + 
          f"rmse: {run.data.metrics['rmse']:.3f}")

Run with the best (lower) RMSE: 
 run id: 2c2f2159b2c34d97832145de412b2130, 
 rmse: 5.335


## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* 5.567
* 6.061
* 6.568

In [33]:
# Acess Experiment by name
experiment_name = "random-forest-best-models"
experiment = client.get_experiment_by_name(experiment_name)

# Searching the best run
best_run = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.test_rmse ASC"]
)[0]

best_rmse = best_run.data.metrics['test_rmse']
print(f"RMSE for the best model: {best_rmse:.3f}")

RMSE for the best model: 5.567
